In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 20      # number of variables
num_ineq = 20     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_exact_20-20.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:27:39<00:00,  5.26s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     -5.109099               0.0      5.256546
std       0.323551               0.0      2.600641
min      -6.006067               0.0      0.735355
25%      -5.343451               0.0      3.391160
50%      -5.115216               0.0      4.841972
75%      -4.872300               0.0      6.673741
max      -4.041517               0.0     16.376632
Number of infeasible solution: 0


In [7]:
df = pd.read_csv("result/cq_exact_20-20.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    -5.120363               0.0      5.415370
std      0.343169               0.0      2.522497
min     -5.935736               0.0      1.261963
25%     -5.368253               0.0      3.348413
50%     -5.130437               0.0      5.240330
75%     -4.863931               0.0      6.869727
max     -4.282859               0.0     12.718379
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_20-20.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:44<00:00,  6.08it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -5.158872          0.439137      0.149969
std       0.330777          0.199082      0.145878
min      -6.150297          0.000000      0.121147
25%      -5.386111          0.297969      0.127323
50%      -5.172906          0.417843      0.129306
75%      -4.928006          0.566141      0.140828
max      -4.087857          1.143811      3.696838
Number of infeasible solution: 997


In [8]:
df = pd.read_csv("result/cq_heur_rnd_20-20.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -5.179348          0.437249      0.130713
std      0.351804          0.205353      0.010380
min     -5.960970          0.031061      0.123118
25%     -5.419368          0.304010      0.126733
50%     -5.216537          0.419553      0.127920
75%     -4.909583          0.607390      0.129207
max     -4.300917          1.030549      0.197645
Number of infeasible solution: 100


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_20-20.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:13<00:00,  7.46it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   4.389183e+10               0.0      0.131604
std    1.380190e+12               0.0      0.012561
min   -1.947171e+00               0.0      0.118814
25%    1.039965e+00               0.0      0.122474
50%    9.580554e+00               0.0      0.134600
75%    4.019358e+02               0.0      0.137523
max    4.364520e+13               0.0      0.296020
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 595.89
Epoch 1, Validation Loss: -0.41
Epoch 2, Validation Loss: -2.17
Epoch 3, Validation Loss: -2.30
Epoch 4, Validation Loss: -1.58
Epoch 5, Validation Loss: -2.63
Epoch 6, Validation Loss: -2.29
Epoch 7, Validation Loss: -2.73
Epoch 8, Validation Loss: -2.67
Epoch 9, Validation Loss: -2.92
Epoch 10, Validation Loss: -2.74
Epoch 11, Validation Loss: -2.70
Epoch 12, Validation Loss: -2.81
Epoch 13, Validation Loss: -2.46
Epoch 14, Validation Loss: -2.65
Epoch 15, Validation Loss: -2.46
Epoch 16, Validation Loss: -2.58
Epoch 17, Validation Loss: -2.31
Epoch 18, Validation Loss: -0.62
Epoch 19, Validation Loss: -2.93
Epoch 20, Validation Loss: -2.78
Epoch 21, Validation Loss: -2.68
Epoch 22, Validation Loss: -3.05
Epoch 23, Validation Loss: -3.20
Epoch 24, Validation Loss: -3.19
Epoch 25, Validation Loss: -2.88
Epoch 26, Validation Loss: -3.26
Epoch 27, Validation Loss: -2.90
Epoch 28, Validation Loss: -3.08
Epoch 29, Validation Loss: -3.02
Epoch 30, Validatio

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 133.32it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -4.147645          0.002640      0.003835
std      0.402367          0.014346      0.001390
min     -4.905395          0.000000      0.001506
25%     -4.421822          0.000000      0.003000
50%     -4.193520          0.000000      0.003521
75%     -3.849404          0.000000      0.004545
max     -3.103360          0.106557      0.008517
Number of infeasible solution: 4


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 492.62
Epoch 1, Validation Loss: -0.27
Epoch 2, Validation Loss: -2.51
Epoch 3, Validation Loss: -2.81
Epoch 4, Validation Loss: -3.09
Epoch 5, Validation Loss: -2.84
Epoch 6, Validation Loss: -3.13
Epoch 7, Validation Loss: -2.75
Epoch 8, Validation Loss: -3.03
Epoch 9, Validation Loss: -2.98
Epoch 10, Validation Loss: -3.12
Epoch 11, Validation Loss: -3.07
Epoch 12, Validation Loss: -2.77
Epoch 13, Validation Loss: -3.23
Epoch 14, Validation Loss: -3.03
Epoch 15, Validation Loss: -3.29
Epoch 16, Validation Loss: -3.26
Epoch 17, Validation Loss: -3.10
Epoch 18, Validation Loss: -1.34
Epoch 19, Validation Loss: -2.84
Epoch 20, Validation Loss: -3.33
Epoch 21, Validation Loss: -2.06
Epoch 22, Validation Loss: -2.72
Epoch 23, Validation Loss: -3.46
Epoch 24, Validation Loss: -3.45
Epoch 25, Validation Loss: -3.46
Epoch 26, Validation Loss: -3.48
Epoch 27, Validation Loss: -3.66
Epoch 28, Validation Loss: -3.55
Epoch 29, Validation Loss: -3.53
Epoch 30, Validatio

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 120.94it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -4.328512          0.001035      0.003753
std      0.319526          0.010352      0.000964
min     -5.053879          0.000000      0.002003
25%     -4.550607          0.000000      0.003003
50%     -4.341492          0.000000      0.003526
75%     -4.121362          0.000000      0.004457
max     -3.587453          0.103516      0.008071
Number of infeasible solution: 1


### Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 494.47
Epoch 1, Validation Loss: -0.44
Epoch 2, Validation Loss: -2.54
Epoch 3, Validation Loss: -3.12
Epoch 4, Validation Loss: -3.20
Epoch 5, Validation Loss: -3.43
Epoch 6, Validation Loss: -3.32
Epoch 7, Validation Loss: -3.43
Epoch 8, Validation Loss: -3.32
Epoch 9, Validation Loss: -3.40
Epoch 10, Validation Loss: -3.40
Epoch 11, Validation Loss: -3.24
Epoch 12, Validation Loss: -3.42
Epoch 13, Validation Loss: -3.15
Epoch 14, Validation Loss: -3.46
Epoch 15, Validation Loss: -3.65
Epoch 16, Validation Loss: -3.73
Epoch 17, Validation Loss: -3.67
Epoch 18, Validation Loss: -3.70
Epoch 19, Validation Loss: -3.74
Epoch 20, Validation Loss: -3.68
Epoch 21, Validation Loss: -3.82
Epoch 22, Validation Loss: -3.91
Epoch 23, Validation Loss: -3.85
Epoch 24, Validation Loss: -3.94
Epoch 25, Validation Loss: -3.72
Epoch 26, Validation Loss: -3.94
Epoch 27, Validation Loss: -3.87
Epoch 28, Validation Loss: -3.92
Epoch 29, Validation Loss: -3.96
Epoch 30, Validatio

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_pr_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 208.69it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -4.659793          0.042595      0.000707
std      0.355136          0.066922      0.000559
min     -5.371934          0.000000      0.000000
25%     -4.935126          0.000000      0.000000
50%     -4.684525          0.005193      0.000999
75%     -4.447333          0.059425      0.001006
max     -3.870809          0.337029      0.002042
Number of infeasible solution: 52
